In [ ]:
! pip install -r requirements.txt

### 1. 產生key pair 

In [ ]:
from joserfc import jwk

In [ ]:
key = jwk.generate_key("RSA", 2048)

In [ ]:
import json

# Public key with kid and alg
public_key = key.as_dict(private=False)
public_key["kid"] = key.thumbprint()
public_key["alg"] = "RS256"
print(json.dumps(public_key))

In [ ]:
# Private key with kid and alg
private_key = key.as_dict()
private_key["kid"] = key.thumbprint()
private_key["alg"] = "RS256"
print(json.dumps(private_key))

### 2. Retrieve .well-known/smart-configuration

In [ ]:
import requests

FHIR_BASE_URL = "https://launch.smarthealthit.org/v/r4/sim/WzQsIiIsIiIsIiIsMCwwLDAsIiIsIiIsIiIsIiIsIiIsIiIsIntcImtleXNcIjpbe1wiblwiOiBcIm56eC1tODI4OXlUdUc0eFNJcGs4RjJad3JxLTFxclZyZFU4ZE5JZGV3endxZ3FJRUw1ZHRYZG53Mmo4Q1VHMzNWblNBNmtJLW9RRmFmVG02dHFBSTZweGRFV1NDcHExYXBKR0xBUURBRlV3bzE1UFMySjI3S1FsWjhjUTA1S1Znd2NESGI1LWlNZFpxX29aVzRCYl9WTGtoWnJsOWFjSE9qRTRPdHl6bDMzVU8wTzNpcHpDUEIxVmUxazN1Y2hzWDRpaHFzNk1BQUV3VlNMU0ExX0xVM1BKbkhyd1dlcTJhQW5SNi05cGFLZ1ZmQl9DSzhua3ZQTWV5cTdMTnpTZUVNU0I3Z2tJbnBiMU1LdEFILW1QNnJiVG1BbXA0Z2o5UWdwVUNTN0hHQlpPanpJaTZkZDAwM0FUWi1rT0FRMl81NHk2UHB5RXNRNXlLcVVMRVZVTjNjUVwiLCBcImVcIjogXCJBUUFCXCIsIFwia3R5XCI6IFwiUlNBXCIsIFwia2lkXCI6IFwiT0lQZWZxeEZxeHFhZWk0cmM4ZDNwNDNuZUhxQXYxYUNSQjRMcW5PTUd6a1wiLCBcImFsZ1wiOiBcIlJTMjU2XCJ9XG5dfSIsMCwxLCIiXQ/fhir"
Client_ID = "ecd805fd-211a-4586-ad03-7c866bc837d7"

In [ ]:
resp = requests.get(f"{FHIR_BASE_URL}/.well-known/smart-configuration")
smart_configuration = resp.json()
smart_configuration

### 3. Create Client Assertion Token

In [ ]:
from joserfc import jwt
import datetime

In [ ]:
header = {
    "alg": "RS256",
    "typ": "JWT",
    "kid": key.thumbprint(),
}

now = datetime.datetime.now(datetime.UTC)
claims = {
    "iss": Client_ID,
    "sub": Client_ID,
    "aud": smart_configuration['token_endpoint'],
    "exp": now + datetime.timedelta(minutes=5),
    "jti": "random-non-reusable-jwt-id-123"
}

client_assertion = jwt.encode(header, claims, key, )
print(client_assertion)

### 4. Retrieve Access Token

In [ ]:
resp = requests.post(
    smart_configuration['token_endpoint'],
    data={
        "grant_type": "client_credentials",
        "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
        "client_assertion": client_assertion,
        "scope": "system/*.*"
    },
    headers={"Content-Type": "application/x-www-form-urlencoded"}
)

token = resp.json().get("access_token")

resp.json()

### 5. Access FHIR Resource

In [ ]:
resp = requests.get(
    f"{FHIR_BASE_URL}/Patient",
    headers={
        "Authorization": f"Bearer {token}"
    }
)
resp.json()